conda env create -n dnn_cpu python=3.12 numpy pandas scikit-learn tensorflow-cpu

TODO readme

In [6]:
import pickle
with open("../data/tmp/ml_data_at_sugaramino.pickle", "rb") as handle:
    ml_datasets = pickle.load(handle)

In [ ]:
feature_name, X, y, sample_names, feature_names = ml_datasets[11]

In [ ]:
from tensorflow import keras

def create_model(input_shape):
    # TODO possibly more layers, try without dropout
    model = keras.Sequential(
        [
            keras.layers.Input(shape=(input_shape,)),
            keras.layers.Dense(512, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(256, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(128, activation="relu"),
            keras.layers.Dense(1, activation="sigmoid"),
        ]
    )

    model.compile(
        optimizer="adam",  # Adam(learning_rate=0.001)
        loss="binary_crossentropy",
        metrics=["Precision", "Recall", "AUC"],
    )  # ['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])
    #     metrics=['Precision', 'Recall', 'AUC']

    # TODO class weights
    return model

model = create_model(len(feature_names))

In [ ]:
# from sklearn.utils import class_weight
# import numpy as np

# class_weights = class_weight.compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(y_train),
#     y=y_train
# )
# class_weights_dict = dict(enumerate(class_weights))

# model.fit(X_train, y_train, 
#           epochs=20, 
#           batch_size=32, 
#           class_weight=class_weights_dict, 
#           validation_data=(X_val, y_val))

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(0)
tf.random.set_seed(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    keras.layers.Input(shape=(len(feature_names),)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["Accuracy","Precision", "Recall", "AUC"])

history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

res = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {test_acc:.4f}")
print(res)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - AUC: 0.5333 - Accuracy: 0.5000 - Precision: 0.5500 - Recall: 0.5789 - loss: 0.7314 - val_AUC: 1.0000 - val_Accuracy: 0.5556 - val_Precision: 0.3333 - val_Recall: 1.0000 - val_loss: 0.6070
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - AUC: 0.7719 - Accuracy: 0.6765 - Precision: 0.6667 - Recall: 0.8421 - loss: 0.5635 - val_AUC: 1.0000 - val_Accuracy: 0.8889 - val_Precision: 0.6667 - val_Recall: 1.0000 - val_loss: 0.5255
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - AUC: 0.8158 - Accuracy: 0.7059 - Precision: 0.7143 - Recall: 0.7895 - loss: 0.5286 - val_AUC: 1.0000 - val_Accuracy: 0.8889 - val_Precision: 0.6667 - val_Recall: 1.0000 - val_loss: 0.4587
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - AUC: 0.9649 - Accuracy: 0.9412 - Precision: 0.9474 - Recall: 0.9474 - loss: 0.3510 - val_AUC: 1.0000 - val_Accuracy: 0.8889 - val_Precision: 0.6667 - val_Recall: 1.0000 - val_loss: 0.4088
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52